# 🏃🏼‍♂️ FastHerbie

Often, data from several GRIB2 files is needed (range of datetimes and/or forecast lead time). `FastHerbie()` use multithreading to help you efficiently create multiple Herbie objects. Then you can download many files using multiprocessing. When you open the data with xarray, it will concatenate the DataSets for a range of model runs and forecast lead times.

In this example, we will get the F00-F02 forecasts for each of the runs initialized between 00z-02z on January 1, 2022 (a total of 9 Herbie objects).


In [1]:
from herbie import FastHerbie
import pandas as pd

In [7]:
# Create a range of dates
DATES = pd.date_range(
    start="2022-03-01 00:00",
    periods=3,
    freq="1H",
)

# Create a range of forecast lead times
fxx = range(0, 3)

In [8]:
# Make FastHerbie Object.
FH = FastHerbie(DATES, model="hrrr", fxx=fxx)
FH

At it's core, ``FastHerbie`` uses multithreading to make a list of Herbie objects. The list of Herbie objects is stored in the `objects` property.

In [9]:
FH.objects

[▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 00:00 UTC F00 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 00:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 00:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 01:00 UTC F00 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 01:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 01:00 UTC F02 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 02:00 UTC F00 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 02:00 UTC F01 ┊ source=aws,
 ▌▌Herbie HRRR model sfc product initialized 2022-Mar-01 02:00 UTC F02 ┊ source=aws]

You can download those Herbie objects as full files or subsets.

In [7]:
# Download full file
FH.download()

In [ ]:
# Download subset
FH.download("TMP:2 m")

You can also read the data with xarray

In [11]:
ds = FH.xarray("TMP:2 m", remove_grib=False)
ds

👨🏻‍🏭 Created directory: [C:\Users\blaylock\data\hrrr\20220301]


<xarray.Dataset>
Dimensions:              (time: 3, step: 3, y: 1059, x: 1799)
Coordinates:
  * time                 (time) datetime64[ns] 2022-03-01 ... 2022-03-01T02:0...
  * step                 (step) timedelta64[ns] 00:00:00 01:00:00 02:00:00
    heightAboveGround    float64 2.0
    latitude             (y, x) float64 21.14 21.15 21.15 ... 47.86 47.85 47.84
    longitude            (y, x) float64 237.3 237.3 237.3 ... 299.0 299.0 299.1
    valid_time           (step, time) datetime64[ns] 2022-03-01 ... 2022-03-0...
Dimensions without coordinates: y, x
Data variables:
    t2m                  (step, time, y, x) float32 292.7 292.7 ... 263.4 263.4
    gribfile_projection  object None
Attributes:
    GRIB_edition:            2
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    model:                   hrrr
    product:                 sfc
    description:             High-Resolution Rapid Refresh - CONUS
    search:            TMP:2 m

## Example using the `preprocess` keyword argument
The `preprocess` keyword argument is useful when you want to trim or modify each dataset before it’s loaded into memory, which can significantly reduce memory usage and improve performance. For example, you might use it to select a geographic subset with .sel(), drop unnecessary variables, or apply chunking. This is especially helpful when working with large GRIB files or combining many datasets, as it ensures only the relevant data is loaded and processed.

In [25]:
# Define a preprocess function that selects a lat/lon region
def trim_domain(ds):
    return ds.sel(
        latitude=slice(50, 30),
        longitude=slice(-130, -90)
    )

In [26]:
# Create a FastHerbie object
fh = FastHerbie(["2025-07-02"], model="ifs", fxx=[0,6,12], max_threads=12)
fh.inventory(":2t")

,grib_message,start_byte,end_byte,range,reference_time,valid_time,step,param,levelist,levtype,number,domain,expver,class,type,stream,search_this,FILE
0,111,75967701,76623282,75967701-76623282,2025-07-02,2025-07-02 00:00:00,0 days 00:00:00,2t,NaN,sfc,NaN,g,0001,od,fc,oper,:2t:sfc:g:0001:od:fc:oper,https://ecmwf-forecasts.s3.eu-central-1.amazon...
1,79,63061141,63717705,63061141-63717705,2025-07-02,2025-07-02 06:00:00,0 days 06:00:00,2t,NaN,sfc,NaN,g,0001,od,fc,oper,:2t:sfc:g:0001:od:fc:oper,https://ecmwf-forecasts.s3.eu-central-1.amazon...
2,84,67243339,67905783,67243339-67905783,2025-07-02,2025-07-02 12:00:00,0 days 12:00:00,2t,NaN,sfc,NaN,g,0001,od,fc,oper,:2t:sfc:g:0001:od:fc:oper,https://ecmwf-forecasts.s3.eu-central-1.amazon...


In [27]:
ds = fh.xarray(":2t", preprocess=trim_domain)

In [28]:
ds

<xarray.Dataset> Size: 158kB
Dimensions:              (step: 3, latitude: 81, longitude: 161)
Coordinates:
    time                 datetime64[ns] 8B 2025-07-02
  * step                 (step) timedelta64[ns] 24B 00:00:00 06:00:00 12:00:00
    heightAboveGround    float64 8B 2.0
  * latitude             (latitude) float64 648B 50.0 49.75 49.5 ... 30.25 30.0
  * longitude            (longitude) float64 1kB -130.0 -129.8 ... -90.25 -90.0
    valid_time           (step) datetime64[ns] 24B 2025-07-02 ... 2025-07-02T...
    gribfile_projection  object 8B None
Data variables:
    t2m                  (step, latitude, longitude) float32 156kB 285.6 ... ...
Attributes:
    GRIB_edition:            2
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    model:                   ifs
    product:                 oper
    description:             ECMWF Open Data - Integrated Forecast System
    search:                  :2t